In [ ]:
# Using this workbook:

For interactive testing execute the first block to get things set up then use the next block to test different ideas for functions by replacing the function being worked on.

For simplicity I just copied the entire script and split it into two cells.

In [1]:
import pandas as pd

stats = pd.read_csv('bullscowsstats.csv')



In [2]:
print(stats)

                 gametime    player  positions  range           answer  \
0     2021/05/12 16:07:13  computer          3     15       [0, 12, 5]   
1     2021/05/12 16:08:26  computer          3     15       [0, 12, 5]   
2     2021/05/12 16:10:27  computer          4      7     [0, 4, 5, 1]   
3     2021/05/12 16:10:45  computer          4      7     [0, 4, 5, 1]   
4     2021/05/12 16:11:10  computer          4      7     [0, 4, 5, 1]   
...                   ...       ...        ...    ...              ...   
2467  2021/05/13 15:23:50  computer          4      7     [0, 4, 5, 1]   
2468  2021/05/13 15:23:52  computer          4      7     [0, 4, 5, 1]   
2469  2021/05/13 15:29:00  computer          4      7     [0, 4, 5, 1]   
2470  2021/05/13 15:35:39  computer          4      7     [0, 4, 5, 1]   
2471  2021/05/13 15:42:28  computer          5      7  [0, 4, 5, 1, 0]   

      guesses  bull_weight  cow_weight  all_factor  guesses_generated  \
0           9          0.5         0.3

In [6]:
dice_df_4x7 = stats[(stats["range"] == 7) & (stats["positions"] == 4)]
print(dice_df_4x7)


                 gametime    player  positions  range        answer  guesses  \
2     2021/05/12 16:10:27  computer          4      7  [0, 4, 5, 1]        5   
3     2021/05/12 16:10:45  computer          4      7  [0, 4, 5, 1]        5   
4     2021/05/12 16:11:10  computer          4      7  [0, 4, 5, 1]        6   
5     2021/05/12 16:11:22  computer          4      7  [0, 4, 5, 1]        5   
6     2021/05/12 16:11:35  computer          4      7  [0, 4, 5, 1]        6   
...                   ...       ...        ...    ...           ...      ...   
2466  2021/05/13 15:23:32  computer          4      7  [0, 4, 5, 1]        5   
2467  2021/05/13 15:23:50  computer          4      7  [0, 4, 5, 1]        5   
2468  2021/05/13 15:23:52  computer          4      7  [0, 4, 5, 1]        5   
2469  2021/05/13 15:29:00  computer          4      7  [0, 4, 5, 1]        5   
2470  2021/05/13 15:35:39  computer          4      7  [0, 4, 5, 1]        6   

      bull_weight  cow_weight  all_fact

In [9]:
avggs = dice_df_4x7['guesses'].mean() 
print("average guesses: {}".format(avggs))

average guesses: 5.056955093099671


In [34]:
grpavg = stats[(stats["range"] == 7) & (stats["player"] == 'computer')].groupby(["positions","bull_weight","cow_weight"]).mean()
print(grpavg)


                                  range   guesses  all_factor  \
positions bull_weight cow_weight                                
2         0.5         0.3           7.0  4.455000         0.5   
3         0.5         0.3           7.0  4.524444         0.5   
                      0.5           7.0  4.453333         0.5   
          0.7         0.3           7.0  4.373333         0.5   
4         0.5         0.3           7.0  5.027451         0.5   
                      0.5           7.0  4.985000         0.5   
          0.7         0.3           7.0  5.104803         0.5   
5         0.5         0.3           7.0  5.533898         0.5   
          0.7         0.3           7.0  5.545455         0.5   
6         0.5         0.3           7.0  5.916667         0.5   

                                  guesses_generated  duplicate_guesses  \
positions bull_weight cow_weight                                         
2         0.5         0.3                 18.020000           1.465000 

In [14]:
%matplotlib

Matplotlib is building the font cache; this may take a moment.
Using matplotlib backend: MacOSX


In [15]:
grpavg.plot()

<AxesSubplot:xlabel='positions'>

In [20]:
print(stats[(stats["range"] == 7)].groupby("positions").describe())

            range                                    guesses            ...  \
            count mean  std  min  25%  50%  75%  max   count      mean  ...   
positions                                                               ...   
2           200.0  7.0  0.0  7.0  7.0  7.0  7.0  7.0   200.0  4.455000  ...   
3          1050.0  7.0  0.0  7.0  7.0  7.0  7.0  7.0  1050.0  4.460952  ...   
4           913.0  7.0  0.0  7.0  7.0  7.0  7.0  7.0   913.0  5.056955  ...   
5           228.0  7.0  0.0  7.0  7.0  7.0  7.0  7.0   228.0  5.539474  ...   
6            24.0  7.0  0.0  7.0  7.0  7.0  7.0  7.0    24.0  5.916667  ...   

          clue_bulls0cowsA      c_bullscowsA                                \
                       75%  max        count      mean       std  min  25%   
positions                                                                    
2                      0.0  1.0        200.0  0.120000  0.325777  0.0  0.0   
3                      0.0  0.0       1050.0  0.231429 

In [19]:
print(stats[(stats["range"] == 7)].groupby("positions").mean())

           range   guesses  bull_weight  cow_weight  all_factor  \
positions                                                         
2            7.0  4.455000     0.500000    0.300000         0.5   
3            7.0  4.460952     0.557143    0.357143         0.5   
4            7.0  5.056955     0.600329    0.343812         0.5   
5            7.0  5.539474     0.596491    0.300000         0.5   
6            7.0  5.916667     0.500000    0.300000         0.5   

           guesses_generated  duplicate_guesses  inconsistent_guesses  \
positions                                                               
2                  18.020000           1.465000             34.015000   
3                  37.212381           1.663810             62.008571   
4                 803.941950           2.301205           1882.808324   
5                4315.614035           2.653509          15496.561404   
6               20862.041667           2.958333          88135.083333   

           clue_bl

In [8]:
print(stats[(stats["range"] == 7) & (stats["positions"] == 4)].describe())


       positions  range     guesses  bull_weight  cow_weight  all_factor  \
count      913.0  913.0  913.000000   913.000000  913.000000       913.0   
mean         4.0    7.0    5.056955     0.600329    0.343812         0.5   
std          0.0    0.0    0.880489     0.100054    0.082767         0.0   
min          4.0    7.0    2.000000     0.500000    0.300000         0.5   
25%          4.0    7.0    5.000000     0.500000    0.300000         0.5   
50%          4.0    7.0    5.000000     0.700000    0.300000         0.5   
75%          4.0    7.0    6.000000     0.700000    0.300000         0.5   
max          4.0    7.0    8.000000     0.700000    0.500000         0.5   

       guesses_generated  duplicate_guesses  inconsistent_guesses  \
count         913.000000         913.000000            913.000000   
mean          803.941950           2.301205           1882.808324   
std          1103.996251           3.240737           3046.103964   
min             1.000000           0.00

In [7]:
print(dice_df_4x7.describe())


       positions  range     guesses  bull_weight  cow_weight  all_factor  \
count      913.0  913.0  913.000000   913.000000  913.000000       913.0   
mean         4.0    7.0    5.056955     0.600329    0.343812         0.5   
std          0.0    0.0    0.880489     0.100054    0.082767         0.0   
min          4.0    7.0    2.000000     0.500000    0.300000         0.5   
25%          4.0    7.0    5.000000     0.500000    0.300000         0.5   
50%          4.0    7.0    5.000000     0.700000    0.300000         0.5   
75%          4.0    7.0    6.000000     0.700000    0.300000         0.5   
max          4.0    7.0    8.000000     0.700000    0.500000         0.5   

       guesses_generated  duplicate_guesses  inconsistent_guesses  \
count         913.000000         913.000000            913.000000   
mean          803.941950           2.301205           1882.808324   
std          1103.996251           3.240737           3046.103964   
min             1.000000           0.00

In [5]:
print(dice_df_4x7)


[['2021/05/12 16:10:27' 'computer' 4 ... 4 0 0]
 ['2021/05/12 16:10:45' 'computer' 4 ... 3 0 0]
 ['2021/05/12 16:11:10' 'computer' 4 ... 3 0 1]
 ...
 ['2021/05/13 15:23:52' 'computer' 4 ... 3 0 0]
 ['2021/05/13 15:29:00' 'computer' 4 ... 2 0 1]
 ['2021/05/13 15:35:39' 'computer' 4 ... 3 0 2]]
